In [74]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import aiohttp
from renetti.ws.spiders.utils import parse_product_json_ld_from_page

In [72]:
url = "https://www.lifefitness.com/en-us/catalog"

In [68]:
async with async_playwright() as playwright:
    async with await playwright.chromium.launch(headless=False) as browser:
        async with await browser.new_context() as context:
            async with await context.new_page() as page:
                page_number = 1
                urls = []
                while True:
                    await page.goto(url=f"{url}/?pageNumber={page_number}#searchform")
                    await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
                    html = await page.content()
                    soup = BeautifulSoup(markup=html)
                    urls += [f"https://www.lifefitness.com{u.get("href")}" for u in soup.findAll("a", class_="product-grid--item")]
                    try:
                        await page.wait_for_selector(f'a[title="Next"]', timeout=2000)
                    except Exception:
                        break
                    page_number += 1


In [75]:
url = "https://www.lifefitness.com/en-us/catalog/cardio/treadmills/integrity-plus"
async with aiohttp.ClientSession() as session:
    async with session.get(url=url) as response:
                html = await response.text()
                soup = BeautifulSoup(markup=html)
res = parse_product_json_ld_from_page(soup=soup)
res

{'name': 'Integrity&#x2B; Treadmill',
 'image_links': ['https://www.lifefitness.com/Kentico13CoreBase/media/LFMedia/LifeFitnessImages/Equipment/Cardio/Integrity%20Plus/Integrity-Treadmill-with-SE4-console-24-inch-in-Black-Onyx.png?ext=.png'],
 'mpn': None,
 'description': '&lt;p&gt;Introducing the next generation of the popular Life Fitness workhorse -- the Integrity&#x2B; Treadmill. Designed to elevate your fitness facility with modern design touches that lend to an inviting aesthetic, and&amp;nbsp;the immersive Discover SE4 Console.&lt;/p&gt;&#xD;&#xA;&#xD;&#xA;&lt;p&gt;More than 500 fitness experiences are available through the complimentary Life Fitness On Demand&#x2B;, available on the SE4 console.&lt;/p&gt;&#xD;&#xA;',
 'brands': ['life fitness'],
 'categories': [],
 'skus': []}

In [77]:
soup.find_all("script", type="application/ld+json")

[<script type="application/ld+json">
         {
           "@context": "https://schema.org",
           "@type": "Product",
           "description": "&lt;p&gt;Introducing the next generation of the popular Life Fitness workhorse -- the Integrity&#x2B; Treadmill. Designed to elevate your fitness facility with modern design touches that lend to an inviting aesthetic, and&amp;nbsp;the immersive Discover SE4 Console.&lt;/p&gt;&#xD;&#xA;&#xD;&#xA;&lt;p&gt;More than 500 fitness experiences are available through the complimentary Life Fitness On Demand&#x2B;, available on the SE4 console.&lt;/p&gt;&#xD;&#xA;",
           "brand": {
               "@type": "Brand",
               "name": "life fitness"
           },
           "name": "Integrity&#x2B; Treadmill",
           "image": "https://www.lifefitness.com/Kentico13CoreBase/media/LFMedia/LifeFitnessImages/Equipment/Cardio/Integrity%20Plus/Integrity-Treadmill-with-SE4-console-24-inch-in-Black-Onyx.png?ext=.png"
         }
     </script>]